In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_columns', None)

import os


username = "root"
password = "Totodile561!"  
host = 'localhost'  
port = '3306'       
database = 'football_db'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

tables = engine.table_names()

player_tables = [table for table in tables if "player" in table]
dfs = {}

for table in player_tables:
    df = pd.read_sql_table(table, con=engine)
    dfs[table] = df
    print(f"Loaded DataFrame for table: {table}")

C:\Users\james\AppData\Local\Temp\ipykernel_17172\2692661351.py:16: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


Loaded DataFrame for table: raw_player_defense_df
Loaded DataFrame for table: raw_player_gca_df
Loaded DataFrame for table: raw_player_keeper_df
Loaded DataFrame for table: raw_player_keepersadv_df
Loaded DataFrame for table: raw_player_passing_df
Loaded DataFrame for table: raw_player_passing_type_df
Loaded DataFrame for table: raw_player_possession_df
Loaded DataFrame for table: raw_player_shooting_df
Loaded DataFrame for table: raw_stats_player_standard_df


In [2]:
dfs.keys()
#dfs["player_defense_df"].head()

dict_keys(['raw_player_defense_df', 'raw_player_gca_df', 'raw_player_keeper_df', 'raw_player_keepersadv_df', 'raw_player_passing_df', 'raw_player_passing_type_df', 'raw_player_possession_df', 'raw_player_shooting_df', 'raw_stats_player_standard_df'])

In [3]:
import hashlib

def make_id(row):
    unique_str = f"{row['Player']}_{row['Nation']}_{row['Born']}"
    return hashlib.md5(unique_str.encode()).hexdigest()

for name, df in dfs.items():
    df['id'] = df.apply(make_id, axis=1)
    dfs[name] = df

In [7]:
drop_rules = {
    'eredivisie': list(range(2010, 2018)),
    'championship': list(range(2010, 2018)),
    'primeira liga': list(range(2010, 2018)),
    'bundesliga': list(range(2010, 2017)),
    'premier league': list(range(2010, 2017)),
    'la liga': list(range(2010, 2017)),
    'serie a': list(range(2010, 2017)),
    'ligue 1': list(range(2010, 2017)),
    'scottish premiership': list(range(2010, 2026))
}

exclude_tables = {
    "raw_stats_player_standard_df",  
    "raw_player_keeper_df",
    "raw_player_shooting_df",   
}
def clean_text_columns(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower()
    return df

def extract_league_and_season(df):
    if 'League' not in df.columns:
        return df

    df[['league_name', 'season']] = df['League'].str.rsplit(' ', n=1, expand=True)
    df['season_start'] = pd.to_numeric(df['season'].str[:4], errors='coerce')
    df = df.dropna(subset=['season_start'])
    df['season_start'] = df['season_start'].astype(int)
    return df

def apply_drop_rules(df, drop_rules):
    if 'league_name' not in df.columns or 'season_start' not in df.columns:
        return df

    mask = df.apply(
        lambda row: row['season_start'] in drop_rules.get(row['league_name'], []),
        axis=1
    )
    return df[~mask]

def process_player_df(df, drop_rules,skip_df=False):
    df = extract_league_and_season(df)
    df = clean_text_columns(df, ['Player', 'Nation', 'Squad', 'League','league_name','Pos'])

    df = df.dropna(subset=['Player', 'Nation', 'Born'], how='all')
    df = df[df['Player'].str.lower() != 'player']

    if not skip_df:
        df = apply_drop_rules(df, drop_rules)
    return df

for key, df in dfs.items():
    if isinstance(df, pd.DataFrame):
        skip = key in exclude_tables
        dfs[key] = process_player_df(df, drop_rules,skip_df=skip)


In [10]:
#dfs["raw_player_defense_df"].tail()
#dfs["raw_player_gca_df"].head(30)



In [6]:

# Count nulls per column
dfs["raw_stats_player_standard_df"].isna().sum()

# Or check percentage of missing values
dfs["raw_stats_player_standard_df"].isna().mean() * 100


Rk               0.000000
Player           0.000000
Nation           0.000000
Pos              0.000000
Squad            0.000000
Age              0.142650
Born             0.136106
MP               0.000000
Starts           0.000000
Min              0.001309
90s              0.000000
Gls              0.001309
Ast              0.043187
G+A              0.043187
G-PK             0.010470
PK               0.010470
PKatt            0.057583
CrdY             0.000000
CrdR             0.000000
xG              52.260800
npxG            52.260800
xAG             52.260800
npxG+xAG        52.260800
PrgC            52.260800
PrgP            52.260800
PrgR            52.260800
Gls.1            0.001309
Ast.1            0.043187
G+A.1            0.043187
G-PK.1           0.010470
G+A-PK           0.052348
xG.1            52.262109
xAG.1           52.262109
xG+xAG          52.262109
npxG.1          52.262109
npxG+xAG.1      52.262109
Matches          6.029237
League           0.000000
id          